<h2 id="toc">목차</h2>

1. 라이브러리 불러오기
2. 데이터 불러오기
3. 파생 컬럼 생성 및 데이터 정제, 변환
4. 컬럼 설명
5. 데이터 저장

<h3 id="contents1">1. 라이브러리 불러오기</h3>

In [ ]:
import pandas as pd
import re

from paths import RAW_DATA, PROCESSED_DATA

<h3 id="contents2">2. 데이터 불러오기</h3>

In [ ]:
# 데이터 
df = pd.read_csv(RAW_DATA, encoding='utf-8-sig')
df

,번호,정답,난이도,배점,정답률,선지1,선지2,선지3,선지4,선지5,시험과목,시험일,시험구분,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex
0,1,5,하,2,91%,2%,2%,2%,3%,91%,확률과통계,2025.05.08,학력평가,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,NaN,NaN
1,2,1,하,2,91%,91%,2%,2%,4%,2%,확률과통계,2025.05.08,학력평가,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,NaN,NaN
2,3,3,중하,3,86%,2%,3%,86%,6%,2%,확률과통계,2025.05.08,학력평가,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,NaN,NaN,NaN
3,4,1,하,3,90%,90%,3%,3%,2%,1%,확률과통계,2025.05.08,학력평가,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,NaN,NaN,NaN
4,5,4,하,3,92%,1%,2%,3%,92%,2%,확률과통계,2025.05.08,학력평가,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,25%,75% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,NaN,NaN,NaN
4736,27,256,상,4,31%,69% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,NaN,NaN,NaN
4737,28,4,중상,4,52%,48% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,NaN,NaN,NaN
4738,29,336,중,4,65%,35% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...",NaN,NaN,NaN


데이터 확인

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740 entries, 0 to 4739
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   번호               4740 non-null   int64  
 1   정답               4740 non-null   int64  
 2   난이도              4740 non-null   object 
 3   배점               4740 non-null   int64  
 4   정답률              4740 non-null   object 
 5   선지1              4740 non-null   object 
 6   선지2              3318 non-null   object 
 7   선지3              3318 non-null   object 
 8   선지4              3318 non-null   object 
 9   선지5              3318 non-null   object 
 10  시험과목             4740 non-null   object 
 11  시험일              4740 non-null   object 
 12  시험구분             4740 non-null   object 
 13  ocr_text_raw     4740 non-null   object 
 14  ocr_text_clean   2 non-null      object 
 15  ocr_text_tokens  0 non-null      float64
 16  ocr_has_latex    0 non-null      float64
dtypes: float64(2),

<h3 id="contents3">3. 파생 컬럼 생성 및 데이터 정제, 변환</h3>

---

'문제유형'컬럼을 추가해줍니다

In [5]:
df["문제유형"] = df["선지1"].str.contains("주관식").map({True: "주관식", False: "객관식"})
df

,번호,정답,난이도,배점,정답률,선지1,선지2,선지3,선지4,선지5,시험과목,시험일,시험구분,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex,문제유형
0,1,5,하,2,91%,2%,2%,2%,3%,91%,확률과통계,2025.05.08,학력평가,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,NaN,NaN,객관식
1,2,1,하,2,91%,91%,2%,2%,4%,2%,확률과통계,2025.05.08,학력평가,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,NaN,NaN,객관식
2,3,3,중하,3,86%,2%,3%,86%,6%,2%,확률과통계,2025.05.08,학력평가,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,NaN,NaN,NaN,객관식
3,4,1,하,3,90%,90%,3%,3%,2%,1%,확률과통계,2025.05.08,학력평가,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,NaN,NaN,NaN,객관식
4,5,4,하,3,92%,1%,2%,3%,92%,2%,확률과통계,2025.05.08,학력평가,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,NaN,NaN,NaN,객관식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,25%,75% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,NaN,NaN,NaN,주관식
4736,27,256,상,4,31%,69% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,NaN,NaN,NaN,주관식
4737,28,4,중상,4,52%,48% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,NaN,NaN,NaN,주관식
4738,29,336,중,4,65%,35% (주관식),NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...",NaN,NaN,NaN,주관식


정답률, 선지1~5 처럼 '%'기호를 포함한 값들은 '%'기호를 빼고

100으로 나눠서 float형으로 바꿔줍시다

In [6]:
percent_col = ['정답률','선지1','선지2','선지3','선지4','선지5']
for col in percent_col:
    if col == '선지1':
        df[col] = df[col].apply(lambda x: float(x.split('%')[0]) / 100 if '주관식' in x else float(x.replace('%', '')) / 100)
    else:
        df[col] = df[col].str.replace('%','').astype(float) / 100
df

,번호,정답,난이도,배점,정답률,선지1,선지2,선지3,선지4,선지5,시험과목,시험일,시험구분,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex,문제유형
0,1,5,하,2,0.91,0.02,0.02,0.02,0.03,0.91,확률과통계,2025.05.08,학력평가,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,NaN,NaN,객관식
1,2,1,하,2,0.91,0.91,0.02,0.02,0.04,0.02,확률과통계,2025.05.08,학력평가,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,NaN,NaN,객관식
2,3,3,중하,3,0.86,0.02,0.03,0.86,0.06,0.02,확률과통계,2025.05.08,학력평가,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,NaN,NaN,NaN,객관식
3,4,1,하,3,0.90,0.90,0.03,0.03,0.02,0.01,확률과통계,2025.05.08,학력평가,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,NaN,NaN,NaN,객관식
4,5,4,하,3,0.92,0.01,0.02,0.03,0.92,0.02,확률과통계,2025.05.08,학력평가,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,NaN,NaN,NaN,객관식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,0.25,0.75,NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,NaN,NaN,NaN,주관식
4736,27,256,상,4,0.31,0.69,NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,NaN,NaN,NaN,주관식
4737,28,4,중상,4,0.52,0.48,NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,NaN,NaN,NaN,주관식
4738,29,336,중,4,0.65,0.35,NaN,NaN,NaN,NaN,수학(나),2016.03.10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...",NaN,NaN,NaN,주관식


'시험일' 컬럼 타입을 datetime으로 바꿔줍시다

In [7]:
df.시험일 = pd.to_datetime(df.시험일, format="%Y.%m.%d")
df

,번호,정답,난이도,배점,정답률,선지1,선지2,선지3,선지4,선지5,시험과목,시험일,시험구분,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex,문제유형
0,1,5,하,2,0.91,0.02,0.02,0.02,0.03,0.91,확률과통계,2025-05-08,학력평가,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,NaN,NaN,객관식
1,2,1,하,2,0.91,0.91,0.02,0.02,0.04,0.02,확률과통계,2025-05-08,학력평가,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,NaN,NaN,객관식
2,3,3,중하,3,0.86,0.02,0.03,0.86,0.06,0.02,확률과통계,2025-05-08,학력평가,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,NaN,NaN,NaN,객관식
3,4,1,하,3,0.90,0.90,0.03,0.03,0.02,0.01,확률과통계,2025-05-08,학력평가,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,NaN,NaN,NaN,객관식
4,5,4,하,3,0.92,0.01,0.02,0.03,0.92,0.02,확률과통계,2025-05-08,학력평가,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,NaN,NaN,NaN,객관식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,0.25,0.75,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,NaN,NaN,NaN,주관식
4736,27,256,상,4,0.31,0.69,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,NaN,NaN,NaN,주관식
4737,28,4,중상,4,0.52,0.48,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,NaN,NaN,NaN,주관식
4738,29,336,중,4,0.65,0.35,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...",NaN,NaN,NaN,주관식


컬럼명에 영어, 한글이 섞여있습니다

컬럼명을 영어로 통일시켜 줍시다

In [8]:
df = df.rename(columns={
    '번호': 'question_number',
    '정답': 'answer',
    '난이도': 'difficulty',
    '배점': 'score',
    '정답률': 'accuracy_rate',
    '선지1': 'option1',
    '선지2': 'option2',
    '선지3': 'option3',
    '선지4': 'option4',
    '선지5': 'option5',
    '시험과목': 'subject',
    '시험일': 'exam_date',
    '시험구분': 'exam_type',
    '문제유형': 'question_type',
    
})
df

,question_number,answer,difficulty,score,accuracy_rate,option1,option2,option3,option4,option5,subject,exam_date,exam_type,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex,question_type
0,1,5,하,2,0.91,0.02,0.02,0.02,0.03,0.91,확률과통계,2025-05-08,학력평가,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,NaN,NaN,객관식
1,2,1,하,2,0.91,0.91,0.02,0.02,0.04,0.02,확률과통계,2025-05-08,학력평가,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,NaN,NaN,객관식
2,3,3,중하,3,0.86,0.02,0.03,0.86,0.06,0.02,확률과통계,2025-05-08,학력평가,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,NaN,NaN,NaN,객관식
3,4,1,하,3,0.90,0.90,0.03,0.03,0.02,0.01,확률과통계,2025-05-08,학력평가,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,NaN,NaN,NaN,객관식
4,5,4,하,3,0.92,0.01,0.02,0.03,0.92,0.02,확률과통계,2025-05-08,학력평가,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,NaN,NaN,NaN,객관식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,0.25,0.75,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,NaN,NaN,NaN,주관식
4736,27,256,상,4,0.31,0.69,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,NaN,NaN,NaN,주관식
4737,28,4,중상,4,0.52,0.48,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,NaN,NaN,NaN,주관식
4738,29,336,중,4,0.65,0.35,NaN,NaN,NaN,NaN,수학(나),2016-03-10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...",NaN,NaN,NaN,주관식


'ocr_text_raw'를 기반으로 비어있는 컬럼들을 채워야합니다

'ocr_text_raw'는 시험문제(문항, 선택지) 전체가 텍스트로 들어있습니다

비어있는 컬럼 목록 : 'ocr_text_clean', 'ocr_text_tokens', 'ocr_has_latex'

---

1. 'ocr_text_clean' 컬럼 완성하기
    - 'ocr_text_raw' 컬럼에서 문제 지문만 추출합니다
    - 선택지('option1 ~ 5')를 제외하고 추출합니다

In [9]:
def extract_problem_including_score(row: pd.Series):
    # 각 행(row)에서 'ocr_text_raw' 컬럼의 텍스트와 'question_type' 값을 가져옵니다.
    text: str = row['ocr_text_raw']
    qtype: str = row['question_type']
    
    if qtype == '주관식':  # 만약 문제 유형이 '주관식'이라면,
        # 선택지가 없을 가능성이 크므로 텍스트를 그대로 반환합니다.
        return text.strip()  # 앞뒤 공백 제거 후 반환

    else:
        # 정규표현식으로 문제 지문 부분만 추출합니다.
        
        # ^               : 문자열 시작부터 탐색
        # (.*?)           : 가능한 적은 문자(최소 매칭)로 그룹 1 캡처
        # \[\d+점\]       : 대괄호 안에 숫자 하나 이상 + '점' 문자 (예: [4점])
        # 문제 지문 끝에 '[숫자점]'까지 포함해서 추출합니다.
        match = re.search(
            r'^(.*?\[\d+점\])', 
            text, 
            re.DOTALL  # re.DOTALL 옵션은 줄바꿈 문자도 '.'이 매칭되게 합니다.
            )

        if match:  # '[숫자점]' 패턴이 발견되면,
            # 매칭된 그룹 1(문제 지문 + 점수 표시)을 가져와서 앞뒤 공백 제거 후 반환
            return match.group(1).strip()
        else:
            # 만약 '[숫자점]' 패턴이 없으면 원본 텍스트 전체를 반환
            return text.strip()

# DataFrame 전체에 행별로 위 함수를 적용하여 'ocr_text_clean'라는 새 컬럼을 만듭니다
df['ocr_text_clean'] = df.apply(extract_problem_including_score, axis=1)

결과 확인

In [10]:
df.loc[:, ['ocr_text_raw', 'ocr_text_clean']].head(10)

,ocr_text_raw,ocr_text_clean
0,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...
1,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...
2,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...
3,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...
4,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...
5,6. \( \frac{3}{2} \pi<\theta<2 \pi \) 인 \( \th...,6. \( \frac{3}{2} \pi<\theta<2 \pi \) 인 \( \th...
6,7. 다항함수 \( f(x) \) 가\n\[\nf^{\prime}(x)=x^{2}-...,7. 다항함수 \( f(x) \) 가\n\[\nf^{\prime}(x)=x^{2}-...
7,8. 부등식 \( 2^{|x|}+\frac{64}{2^{|x|}} \leq 20 \...,8. 부등식 \( 2^{|x|}+\frac{64}{2^{|x|}} \leq 20 \...
8,9. 다항함수 \( f(x) \) 가 모든 실수 \( x \) 에 대하여\n\[\n...,9. 다항함수 \( f(x) \) 가 모든 실수 \( x \) 에 대하여\n\[\n...
9,10. 모든 항이 자연수이고 공차가 같은 두 등차수열 \( \left\{a_{n}\...,10. 모든 항이 자연수이고 공차가 같은 두 등차수열 \( \left\{a_{n}\...


2. 'ocr_text_tokens' 컬럼
    - 1번에서 추출한 'ocr_text_clean' 컬럼을 토큰화

📌 리눅스에서 Mecab 설치 방법 (에러 발생 가능성 높음)

터미널에서 다음 명령어들을 실행해줍니다

```bash
sudo apt-get update
sudo apt-get install -y g++ openjdk-8-jdk python3-dev python3-pip curl
pip install konlpy
sudo apt-get install -y default-jdk
sudo apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
pip install mecab-python3
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip install python-mecab-ko-dic
```

In [11]:
from utils import tokenize_math_text

# 새로운 컬럼 생성
df["ocr_text_tokens"] = df["ocr_text_clean"].apply(tokenize_math_text)

df[["ocr_text_clean", "ocr_text_tokens"]]


,ocr_text_clean,ocr_text_tokens
0,1. \( \left(3^{1-\sqrt{2}}\right)^{2} \times 9...,"[1, ., \left, (, 3, ^, {, 1, -, \sqrt, {, 2, }..."
1,2. 함수 \( f(x)=x^{3}-2 x+5 \) 에 대하여 \( \lim _{h...,"[2, ., 함수, f, (, x, ), =, x, ^, {, 3, }, -, 2,..."
2,3. 첫째항과 공비가 모두 양수 \( k \) 인 등비수열 \( \left\{a_{...,"[3, ., 첫째, 항, 과, 공비, 가, 모두, 양수, k, 인, 등비수열, \l..."
3,4. 함수 \( y=f(x) \) 의 그래프가 그림과 같다.\n\( \lim _{x...,"[4, ., 함수, y, =, f, (, x, ), 의, 그래프, 가, 그림, 과,..."
4,5. 함수 \( f(x)=(2 x+1)\left(x^{2}-2 x+5\right) ...,"[5, ., 함수, f, (, x, ), =, (, 2, x, +, 1, ), \l..."
...,...,...
4735,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,"[26, ., 자연수, n, 에, 대하, 여, 곡선, y, =, x, ^, {, 2..."
4736,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,"[27, ., 전체, 집합, U, =, {, x, \mid, x, 는, 8, 이하,..."
4737,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,"[28, ., 두, 함수, \[, f, (, x, ), =\, left, \{\, ..."
4738,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...","[29, ., 두, 집합, \[, A, =\{, 1, ,, 2, ,, 3, ,, 4..."


3. 'ocr_has_latex' 컬럼
- 'ocr_text_clean'에 '\\'(백슬래시) 문자가 있는지 여부로 판단합니다

In [12]:
# 문제마다 LaTeX 문법이 포함되어 있는지 확인하는 함수
def has_latex(text: str) -> bool:
    if pd.isna(text):
        return False
    return "\\" in text

df["ocr_has_latex"] = df["ocr_text_clean"].apply(has_latex)

In [13]:
df['ocr_has_latex'].value_counts()
df[df["ocr_has_latex"] == True]['ocr_text_tokens']  # LaTeX 문법이 없는 문제들 확인

0       [1, ., \left, (, 3, ^, {, 1, -, \sqrt, {, 2, }...
1       [2, ., 함수, f, (, x, ), =, x, ^, {, 3, }, -, 2,...
2       [3, ., 첫째, 항, 과, 공비, 가, 모두, 양수, k, 인, 등비수열, \l...
3       [4, ., 함수, y, =, f, (, x, ), 의, 그래프, 가, 그림, 과,...
4       [5, ., 함수, f, (, x, ), =, (, 2, x, +, 1, ), \l...
                              ...                        
4735    [26, ., 자연수, n, 에, 대하, 여, 곡선, y, =, x, ^, {, 2...
4736    [27, ., 전체, 집합, U, =, {, x, \mid, x, 는, 8, 이하,...
4737    [28, ., 두, 함수, \[, f, (, x, ), =\, left, \{\, ...
4738    [29, ., 두, 집합, \[, A, =\{, 1, ,, 2, ,, 3, ,, 4...
4739    [30, ., 유리함수, f, (, x, ), =, \frac, {, 8, x, }...
Name: ocr_text_tokens, Length: 4606, dtype: object

결측치를 확인합니다

In [14]:
df.isnull().sum()
# 향후 모델 예측을 위해 '주관식' 문제의 경우 option 컬럼을 0으로 만들어줍시다

question_number       0
answer                0
difficulty            0
score                 0
accuracy_rate         0
option1               0
option2            1422
option3            1422
option4            1422
option5            1422
subject               0
exam_date             0
exam_type             0
ocr_text_raw          0
ocr_text_clean        0
ocr_text_tokens       0
ocr_has_latex         0
question_type         0
dtype: int64

In [15]:
cols = ['option1', 'option2', 'option3', 'option4', 'option5']

# question_type이 '주관식'인 행만 선택해서 0으로 채워줍니다
df.loc[df['question_type'] == '주관식', cols] = 0
df.loc[df['question_type'] == '주관식', :]

,question_number,answer,difficulty,score,accuracy_rate,option1,option2,option3,option4,option5,subject,exam_date,exam_type,ocr_text_raw,ocr_text_clean,ocr_text_tokens,ocr_has_latex,question_type
15,16,10,중하,3,0.86,0.0,0.0,0.0,0.0,0.0,확률과통계,2025-05-08,학력평가,16. 방정식\n\[\n\log _{\sqrt{3}}(x-3)=\log _{3}(5...,16. 방정식\n\[\n\log _{\sqrt{3}}(x-3)=\log _{3}(5...,"[16, ., 방정식, \[, \, log, _{\, sqrt, {, 3, }}(,...",True,주관식
16,17,2,중하,3,0.80,0.0,0.0,0.0,0.0,0.0,확률과통계,2025-05-08,학력평가,17. \( \int_{0}^{a}\left(4 x^{2}-3 x\right) d ...,17. \( \int_{0}^{a}\left(4 x^{2}-3 x\right) d ...,"[17, ., \int, {, 0, }, ^, {, a, }, \left, (, 4...",True,주관식
17,18,23,중하,3,0.83,0.0,0.0,0.0,0.0,0.0,확률과통계,2025-05-08,학력평가,"18. 두 수열 \( \left\{a_{n}\right\},\left\{b_{n}\...","18. 두 수열 \( \left\{a_{n}\right\},\left\{b_{n}\...","[18, ., 두, 수열, \left, {, a, {, n, }, \right, }...",True,주관식
18,19,16,중상,3,0.57,0.0,0.0,0.0,0.0,0.0,확률과통계,2025-05-08,학력평가,19. 최고차항의 계수가 1 인 삼차함수 \( f(x) \) 에 대하여 곡선 \( ...,19. 최고차항의 계수가 1 인 삼차함수 \( f(x) \) 에 대하여 곡선 \( ...,"[19, ., 최고차항, 의, 계수, 가, 1, 인, 삼, 차, 함수, f, (, ...",True,주관식
19,20,110,최상,4,0.19,0.0,0.0,0.0,0.0,0.0,확률과통계,2025-05-08,학력평가,"20. 양수 \( t \) 에 대하여 닫힌구간 \( \left[0, \frac{2}...","20. 양수 \( t \) 에 대하여 닫힌구간 \( \left[0, \frac{2}...","[20, ., 양수, t, 에, 대하, 여, 닫힌구간, \left, 0, \frac...",True,주관식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4735,26,20,최상,4,0.25,0.0,0.0,0.0,0.0,0.0,수학(나),2016-03-10,학력평가,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,26. 자연수 \( n \) 에 대하여 곡선 \( y=x^{2}-\left(4+\f...,"[26, ., 자연수, n, 에, 대하, 여, 곡선, y, =, x, ^, {, 2...",True,주관식
4736,27,256,상,4,0.31,0.0,0.0,0.0,0.0,0.0,수학(나),2016-03-10,학력평가,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,27. 전체집합 \( U=\{x \mid x \) 는 8 이하의 자연수 \( \} ...,"[27, ., 전체, 집합, U, =, {, x, \mid, x, 는, 8, 이하,...",True,주관식
4737,28,4,중상,4,0.52,0.0,0.0,0.0,0.0,0.0,수학(나),2016-03-10,학력평가,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,28. 두 함수\n\[\nf(x)=\left\{\begin{array}{ll}\nx...,"[28, ., 두, 함수, \[, f, (, x, ), =\, left, \{\, ...",True,주관식
4738,29,336,중,4,0.65,0.0,0.0,0.0,0.0,0.0,수학(나),2016-03-10,학력평가,"29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...","29. 두 집합\n\[\nA=\{1,2,3,4\}, B=\{1,2,3,4,5,6,7...","[29, ., 두, 집합, \[, A, =\{, 1, ,, 2, ,, 3, ,, 4...",True,주관식


<h3 id="contents4">4. 컬럼 설명</h3>

---

1.&nbsp;`difficulty` 컬럼은 `accuracy_rate` 컬럼에 의존합니다

| difficulty | accuracy_rate |
| ------ | ----- |
| 최상 | 30% 미만 |
| 상 | 30%~49% |
| 중상 | 50%~59% |
| 중 | 60%~79% |
| 중하 | 80%~89% |
| 하 | 90%~100% |

2.&nbsp;`option1 ~ 5`

- 객관식 문제

    - `option`별 선택 비율을 의미합니다

- **주관식 문제**

    - `option1`은 **오답률**을 의미합니다

    - `option2 ~ 5`는 NaN

3.&nbsp;시험문제지는 홀수형과 짝수형이 존재합니다

- **이번 분석에서는 `홀수형` 시험지만 활용**했습니다

4.&nbsp;`exam_date` : 시험 시행 날짜 (YYYY.MM.DD 형식)

5.&nbsp;`exam_type` : 수능, 모의평가, 학력평가 (고3)

6.&nbsp;`question_type` : 객관식 또는 주관식

7.&nbsp;`subject`

연도별로 `subject` 구분 기준이 상이합니다

| 연도 범위       | `subject` 구분           |
|----------------|--------------------------|
| 2024년 ~ 2021년 | 확률과통계, 미적분, 기하 |
| 2020년 ~ 2016년 | 수학(가), 수학(나)      |

8.&nbsp;`ocr_text_raw`
- OCR에서 추출한 전체 텍스트 원문
- 지문 + 선택지 포함, 줄바꿈 포함

9.&nbsp;`ocr_text_clean`

10.&nbsp;`ocr_text_tokens`

11.&nbsp;`ocr_has_latex`

<h3 id="contents5">5. 데이터 저장</h3>

In [ ]:
# 처음에 한번만 실행
# df.to_csv(PROCESSED_DATA, index=False, encoding="utf-8-sig")